## 文章一：《Retrieval-Augmented Generation for Large Language Models: A Survey（RAG-Survey）》

RAG（Retrieval-Augmented Generation，检索增强生成）在大语言模型（LLM）中的应用，它的核心目的是：

- 减少LLM的“幻觉”（hallucination）现象，即模型生成的内容貌似合理但其实错误。

- 弥补LLM对最新知识、专有领域知识的缺乏。

- 提高LLM在“知识密集型任务”中的表现（如问答、摘要、对话等）。


#### 需要提前学习的基础知识：
- ✅ Transformer架构（强烈推荐从《Attention is All You Need》开始）

- ✅ BERT模型和向量表示

- ✅ 向量检索与相似度计算（如FAISS，余弦相似度）

- ✅ 基础的RAG结构与Prompt构建方式

- ✅ query重写、rerank与context压缩技巧

RAG最典型的流程：

1. Indexing（构建索引）

    - 文本被切分成 chunks，然后使用“嵌入模型”（Embedding Model）将其转化为向量，存入“向量数据库”中。

    - ✅ 【需要学习】嵌入模型（如 BERT embedding）、向量表示、向量数据库（如FAISS）

2. Retrieval（语义检索）

    - 将用户提问转成向量，检索出最相关的Top-k chunks。

    - ✅ 【需要学习】相似度计算（如余弦相似度）、Top-k 检索
3. Generation（生成）

    - 将用户问题和Top-k文档一起输入给LLM生成答案。



### 三大技术范式详解 （目前有五大范式）
1. Naive RAG
    - 最早的RAG形式，优点是简单、有效，缺点是容易出现幻觉、检索不准、回答重复。

    - 问题：

        - 检索阶段选出来的chunks可能与问题无关或信息缺失。

        - 生成阶段可能胡编乱造、不连贯。

    - 扩展上下文时可能冗余重复，风格和逻辑不一致。

2. Advanced RAG
    - 提出了多种优化方法解决上述问题：

        - 预检索优化（Pre-retrieval）

        - 改进 index（滑动窗口切片、加元信息、结构优化）

        - 改进 query（重写query、扩展query、改写语义）

    - 后检索优化（Post-retrieval）

        - rerank检索结果（最相关内容排前）

        - context压缩（减少冗余，突出重点）

    - 需要额外学习：

        - ✅ 【需要学习】query重写技术（例如基于LLM的Rewriter）

        - ✅ 【需要学习】语义搜索和文档重排序技术（如BM25 + rerank）

3. Modular RAG
    - 这是目前最先进的RAG范式，最大的特点是模块化：

    - 引入了新的模块，比如：

        - Search模块：可以使用Google、数据库、知识图谱等。

        - Memory模块：像记忆一样记录对话历史，做强化检索。

        - Routing模块：根据query自动决定从哪类知识中找答案。

        - Predict模块：用于精简上下文，避免冗余。

        - Task Adapter模块：为不同任务生成不同的prompt。

    - 模块化的意义：

        - 支持并行检索、迭代检索、adaptive路径选择等策略。

        - 不再是串行执行“检索→生成”，而是支持更复杂的流程。

### RAG第一范式 - Naive RAG（朴素RAG）

<img src="../picture/img_16.png" width="70%">

“RAG将文档和问题都向量化后，先通过相似度检索找知识，再拼接到Prompt中送入LLM生成答案。”

1. Embedding Model（向量表示模型） —— 图左下角（粉色区域）
- 这一部分负责对文本进行“向量化”，将“文档”或“用户问题”转化为模型可理解的高维向量。

- 🌟 主要流程：

     - Additional Documents → Encode：将预先收集的知识文档（比如PDF、网页、技术手册等）转为统一格式，再输入Embedding模型得到向量。

    - User Query → Encode：用户输入的问题也被编码成一个向量，用于之后的相似性匹配。

- 🔍 技术细节：

    - 通常使用 BERT、RoBERTa、MiniLM 等预训练Transformer模型的最后一层CLS向量或平均池化结果。

    - 输出的是一个固定维度的向量，比如 768 维。

- ✅ 需要学习的基础：
    - 【需要学习】BERT模型：特别是它如何从句子中提取向量

    - 【需要学习】**句子向量（Sentence Embedding）**的提取方式：CLS vs Pooling

2. Vector Database（向量数据库） —— 图中间黄色区域
- 向量化的文档被储存在这里，作为知识库的核心。

- 🌟 两个作用：
    - Index（索引）：将上一步文档的向量存入数据库，支持快速检索。

    - Similarity Search（相似性检索）：用户Query转为向量后，在这里执行“相似度”计算，找到最相关的Top-k文档。

- 🧰 常用工具：
    - FAISS（Facebook AI Similarity Search）

    - Milvus、Weaviate、Qdrant、Pinecone

- ✅ 需要学习的基础：
    - 【需要学习】向量数据库基本结构与查找算法（倒排 vs HNSW等）

    - 【需要学习】相似度度量方法：cosine similarity（余弦相似度）、欧氏距离等
3. Retrieval（检索） —— 图右下角绿色区域
- 这一步是核心桥梁，它把用户Query和找到的Top-k文档结合，生成一个增强版的Prompt。

- 🌟 流程：
    - Query 被拿去查向量数据库，返回“Similar Documents”

    - 把这些 Top-k 文档和原始 Query 拼接成最终 Prompt

- 📌 细节注意：
    - 文档通常是chunk切分后的结果（避免超出LLM上下文长度）

    - 拼接时可以按照重要性排序、添加说明信息等

- ✅ 需要学习的基础：
    - 【需要学习】Prompt 构造方法：拼接顺序、格式控制（如：问题 + 摘要 + 原文片段）

4. Generation（生成） —— 图右上角的 LLM + Response 区域
- 这是你最熟悉的一步：LLM（如GPT）根据Prompt生成答案。

- 🌟 特点：
    - 模型可以是开源的（LLaMA、Mistral）也可以是API（如OpenAI GPT）

    - 如果Prompt设计合理，生成质量能大幅提升

- ✅ 学习建议：
    - 【需要学习】Prompt Engineering（提示词工程）

    - 【需要学习】生成型模型如何在输入上下文中“检索并融合信息”

5. ✅ Naive RAG 的优势与缺陷
- ✅ 优势：
    - 易于搭建：只需要Embedding模型、向量库、LLM三部分。

    - 可更新：知识库（向量库）可以动态更新，不需要重新训练LLM。

    - 实践性强：适用于企业QA、文档问答、合规审核等任务。

- ❌ 缺陷：
    - 依赖检索精度：Top-k文档没找准，LLM回答再好也没用。

    - 容易幻觉：LLM可能胡编，尤其是引用的信息不明确。

    - 不适合复杂推理任务：缺乏对文档间关系的建模（这是GraphRAG和Modular RAG要解决的）

***代码已经在naive_rag_demo中跑通***

### RAG第一范式细节拓展
1. ① 文档切块策略（Chunking）
- 虽然你目前手动写了 5 条简短文段，但现实文档通常很长，需要：

    - 滑动窗口式切块（带 overlap，防止信息割裂）

    - 分段标题辅助切块（章节级别 chunk）

- ✅ 工具包推荐：

    - langchain.text_splitter.RecursiveCharacterTextSplitter

    - unstructured, pdfplumber, PyMuPDF 用于解析文档
2. ② Prompt 模板设计
- 你目前使用了非常基础的：
    - 请根据以下内容回答问题：
    - [文档1] ...
    - [文档2] ...
    - [问题] ...
- 但可以进一步改进：

    - 加入 role 提示：你是法律专家/医生/助理

    - 明确限制回答来源于文档中：不要编造

    - 支持英文/多语种问法

    - 多轮对话结构（加入聊天历史）

- ✅ 可以封装成 Prompt 模板类 PromptBuilder，统一组织。
3. ③ 检索 Top-K 策略调优
- 目前你是默认 top_k=2：`rag = NaiveRAG(top_k=2)`
- 但实际应用中可根据任务调整：
    - top_k 越大：信息覆盖越广，但可能冗余

    - top_k 越小：干净，精度高，但可能缺信息

- ✅ 可选进阶技术：

    - 基于 Similarity Score 设定阈值而非固定 top_k

    - 加入 rerank 阶段（RAG第二范式内容）
4. ④ Embedding 模型选择
- 你目前使用的是：`"all-MiniLM-L6-v2"`
- 非常轻量快速，但也可以考虑以下升级方向：
    - multi-qa-mpnet-base-dot-v1——搜索类任务强
    - bge-base-en-v1.5——中文/英文都强（BGE）
    - intfloat/e5-base——支持指令优化（"query: ... document: ..."）结构
    - text-embedding-3-small（OpenAI）——官方 GPT 嵌入，精度高
5. ⑤ 检索缓存 / 文档ID追踪
- 实际系统中，你需要知道每个检索文档来自哪里：

    - 给每个文档编号、加标签（如来源页码、文件名）

    - 在 Prompt 中提示模型回答时引用具体片段：
        - 回答时请注明“文档X”来源

#### 文档切块（Chunking）
- 引入一个自动化的切块方案，适用于处理任意文本、长文档，未来可以直接对接 PDF、HTML、Word、网页等。
- 目标：将一段或一篇长文本分割为若干段长度适中、语义完整、可被检索的文本块（chunk）
- 步骤：
    - 使用 langchain.text_splitter.RecursiveCharacterTextSplitter 进行切块
    - 设置 chunk_size 和 chunk_overlap 参数，控制块长度和重叠程度
    - 调用 split_text 方法对文本进行切块
    - 返回切块后的列表




# 🔍 RAGPipeline 第一范式全流程实现

In [2]:
## ✨ 1. 引入依赖（确保环境已预安装所需包）
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
from openai import OpenAI

In [3]:
## 🧠 2. 定义 RAGPipeline 类
class RAGPipeline:
    def __init__(self, top_k=3, chunk_size=300, chunk_overlap=50,
                 embedding_model="all-MiniLM-L6-v2",
                 openai_key=None, base_url=None, model="gpt-4"):
        """
            初始化RAGPipeline类，设定检索相关参数、加载嵌入模型和大语言模型客户端。

            参数说明：
            - top_k: 检索时返回最相关的文档片段数量
            - chunk_size: 文本切块时每个块的最大字符数
            - chunk_overlap: 相邻切块之间的重叠字符数
            - embedding_model: 使用的句向量编码模型名称
            - openai_key: OpenAI API 的密钥
            - base_url: OpenAI API的自定义访问地址（适配代理或私有部署）
            - model: 使用的大语言模型名称（如 gpt-4）
        """
        self.top_k = top_k
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.embedding_model = embedding_model
        self.model = model
        self.documents = []

        self.encoder = SentenceTransformer(embedding_model)
        self.client = OpenAI(api_key=openai_key, base_url=base_url)

    def build_knowledge_base_from_text(self, text: str):
        """
        从原始文本构建知识库，包括：文本切块、向量化、FAISS索引构建。

        参数：
        - text: 原始输入文本
        """
        # 初始化文本切块器
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            separators=["\n\n", "\n", ".", "。", "，", ",", " "]
        )
        # 执行切块
        chunks = splitter.split_text(text)
        self.documents = chunks # 保存切片内容
        # 将每个文本块编码成向量
        embeddings = self.encoder.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
        dim = embeddings.shape[1]

        # 构建FAISS索引用于快速检索
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(embeddings)
        self.doc_embeddings = embeddings

        print(f"[✅] 向量库构建完成，{len(chunks)} 个chunk，维度 {dim}")

    def build_prompt(self, query: str, top_chunks: list) -> str:
        """
        根据检索到的文本片段构造Prompt，用于输入到大语言模型中。

        参数：
        - query: 用户的问题
        - top_chunks: 检索出的top-k文本片段
        返回：
        - prompt: 构造好的提示词文本
        """
        prompt = "请根据以下内容回答问题：\n\n"
        for i, chunk in enumerate(top_chunks):
            prompt += f"[文档{i + 1}]:\n{chunk}\n\n"
        prompt += f"[用户问题]:\n{query}\n"
        return prompt

    def retrieve(self, query: str) -> list:
        """
        执行向量检索，获取与用户问题最相似的文档片段。

        参数：
        - query: 用户的问题
        返回：
        - top-k 个相关文档内容组成的列表
        """
        q_vec = self.encoder.encode([query], convert_to_numpy=True)
        distances, indices = self.index.search(q_vec, self.top_k)
        return [self.documents[i] for i in indices[0]]

    def query(self, query: str) -> str:
        """
        主流程：检索+构建prompt+调用语言模型回答。

        参数：
        - query: 用户的问题
        返回：
        - 回答内容或错误信息
        """
        top_docs = self.retrieve(query)
        prompt = self.build_prompt(query, top_docs)
        print("\n[🧾 Prompt 构造结果]\n", prompt)

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "你是一个专业问答助手，请根据文档认真回答用户的问题。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=512
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"❌ Chat生成失败：{e}"


In [4]:
## 📚 3. 构建知识库
text = """
猫是一种常见的家养动物，通常具有敏锐的听觉和视觉，尤其在夜间表现出很强的活动能力。
它们是夜行性动物，常被人们当作宠物饲养。
与猫相比，狗则更具群体性和服从性，常用于看家护院。
狗通常通过训练来完成各种任务，例如搜救、导盲等。
此外，动物与人类的互动在人类社会中发挥着重要作用，不仅提供情感陪伴，还可以协助执行各种任务。
"""

rag = RAGPipeline(
    top_k=2,
    openai_key="sk-tLpNDWa6j2n3MJaGAKQJ67EdtmDLVYndaIylcPASc7N1E6mn",
    base_url="https://chatapi.littlewheat.com/v1",
    model="gpt-3.5-turbo"
)

rag.build_knowledge_base_from_text(text)

[✅] 向量库构建完成，1 个chunk，维度 384


In [5]:
## ❓ 4. 用户提问并生成回答
query = "猫的特点是什么？"
answer = rag.query(query)
print("\n🎯 [回答结果]", answer)


[🧾 Prompt 构造结果]
 请根据以下内容回答问题：

[文档1]:
猫是一种常见的家养动物，通常具有敏锐的听觉和视觉，尤其在夜间表现出很强的活动能力。
它们是夜行性动物，常被人们当作宠物饲养。
与猫相比，狗则更具群体性和服从性，常用于看家护院。
狗通常通过训练来完成各种任务，例如搜救、导盲等。
此外，动物与人类的互动在人类社会中发挥着重要作用，不仅提供情感陪伴，还可以协助执行各种任务。

[文档2]:
猫是一种常见的家养动物，通常具有敏锐的听觉和视觉，尤其在夜间表现出很强的活动能力。
它们是夜行性动物，常被人们当作宠物饲养。
与猫相比，狗则更具群体性和服从性，常用于看家护院。
狗通常通过训练来完成各种任务，例如搜救、导盲等。
此外，动物与人类的互动在人类社会中发挥着重要作用，不仅提供情感陪伴，还可以协助执行各种任务。

[用户问题]:
猫的特点是什么？


🎯 [回答结果] 猫的特点包括：它们是一种常见的家养动物，通常具有敏锐的听觉和视觉，尤其在夜间表现出很强的活动能力。猫是夜行性动物，常被人们当作宠物饲养。
